In [1]:
import os
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data_dir = '/Users/saikarthik/Downloads/untitled folder/parkinsons_dataset'


In [3]:
img_height, img_width = 224, 224 
batch_size = 32


In [4]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)


In [5]:

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
)

Found 665 images belonging to 2 classes.


In [6]:

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
)

Found 166 images belonging to 2 classes.


In [7]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

2025-04-02 11:47:35.249176: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-02 11:47:35.249209: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-02 11:47:35.249218: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-02 11:47:35.249344: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-02 11:47:35.249593: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
)

Epoch 1/100


2025-04-02 11:47:38.732283: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - ETA: 0s - loss: 0.7336 - accuracy: 0.6602

2025-04-02 11:47:45.507479: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 10s 317ms/step - loss: 0.7336 - accuracy: 0.6602 - val_loss: 0.6043 - val_accuracy: 0.7349
Epoch 2/100
21/21 [==============================] - 5s 251ms/step - loss: 0.6520 - accuracy: 0.6767 - val_loss: 0.5739 - val_accuracy: 0.7349
Epoch 3/100
21/21 [==============================] - 5s 242ms/step - loss: 0.6377 - accuracy: 0.7023 - val_loss: 0.5689 - val_accuracy: 0.7349
Epoch 4/100
21/21 [==============================] - 5s 241ms/step - loss: 0.6135 - accuracy: 0.7248 - val_loss: 0.5655 - val_accuracy: 0.7349
Epoch 5/100
21/21 [==============================] - 5s 245ms/step - loss: 0.5871 - accuracy: 0.7278 - val_loss: 0.5591 - val_accuracy: 0.7349
Epoch 6/100
21/21 [==============================] - 5s 241ms/step - loss: 0.5644 - accuracy: 0.7444 - val_loss: 0.5689 - val_accuracy: 0.7349
Epoch 7/100
21/21 [==============================] - 5s 240ms/step - loss: 0.5621 - accuracy: 0.7429 - val_loss: 0.5526 - val_accuracy: 0.7349
Epoch 8/10

In [30]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

validation_generator.reset()
predictions = (model.predict(validation_generator) > 0.5).astype("int64")
true_classes = validation_generator.classes

print("\nClassification Report:")
print(classification_report(true_classes, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(true_classes, predictions))

6/6 [==============================] - 1s 167ms/step - loss: 0.4223 - accuracy: 0.7831
Validation Accuracy: 78.31%
6/6 [==============================] - 1s 175ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.95      0.83       122
           1       0.33      0.07      0.11        44

    accuracy                           0.72       166
   macro avg       0.54      0.51      0.47       166
weighted avg       0.63      0.72      0.64       166


Confusion Matrix:
[[116   6]
 [ 41   3]]
